In [148]:
import pandas as pd
df = pd.read_csv('payments_data_set.csv')
# Убедитесь, что столбец 'Дата оплаты' имеет тип datetime
df['Дата оплаты'] = pd.to_datetime(df['Дата оплаты'])
# Сортировка DataFrame по дате платежа
df_sorted = df.sort_values(by='Дата оплаты')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8438 entries, 0 to 8437
Data columns (total 13 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   Дата оплаты                        8438 non-null   datetime64[ns]
 1   Сумма (тыс. руб)                   8438 non-null   float64       
 2   День недели                        8438 non-null   int64         
 3   Номер рабочего дня в году          8438 non-null   int64         
 4   Номер недели                       8438 non-null   int64         
 5   Год                                8438 non-null   int64         
 6   Сумма платежей с начала недели     8438 non-null   float64       
 7   прошлогодняя сумма на дату         5876 non-null   float64       
 8   прошлогодняя сумма на день         7932 non-null   float64       
 9   прошлогодняя недельная сумма       7958 non-null   float64       
 10  Размер контрактуемых обязательств  8

In [149]:
df = df.dropna()
df = df.drop(df[df['прошлогодняя сумма на дату'] == 0].index)
df = df.drop(df[df['прошлогодняя сумма на день'] == 0].index)
df = df.drop(df[df['прошлогодняя недельная сумма'] == 0].index)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5755 entries, 558 to 8437
Data columns (total 13 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   Дата оплаты                        5755 non-null   datetime64[ns]
 1   Сумма (тыс. руб)                   5755 non-null   float64       
 2   День недели                        5755 non-null   int64         
 3   Номер рабочего дня в году          5755 non-null   int64         
 4   Номер недели                       5755 non-null   int64         
 5   Год                                5755 non-null   int64         
 6   Сумма платежей с начала недели     5755 non-null   float64       
 7   прошлогодняя сумма на дату         5755 non-null   float64       
 8   прошлогодняя сумма на день         5755 non-null   float64       
 9   прошлогодняя недельная сумма       5755 non-null   float64       
 10  Размер контрактуемых обязательств  5755

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 1. Разбиение датафрейма на обучающую и тестовую выборки
# Сортируем датафрейм по дате, чтобы убедиться, что данные в хронологическом порядке
df = df.sort_values(by='Дата оплаты')  # Предполагаем, что 'Дата оплаты' - это название столбца с датами

# Разбиение данных
train_size = int(0.95 * len(df))  # 80% данных для обучения
train, test = df.iloc[:train_size], df.iloc[train_size:]

# 2. Запоминание значений дат для обучающей и тестовой выборки
train_dates = train['Дата оплаты']
test_dates = test['Дата оплаты']

# 3. Запоминание значений целевой переменной для обучающей и тестовой выборки
train_target = train['Сумма (тыс. руб)']
test_target = test['Сумма (тыс. руб)']

In [151]:
import plotly.graph_objects as go
import pandas as pd

# Создаем интерактивный график
fig = go.Figure()
# Общий стиль линий
line_style = dict(width=2)  # сплошная линия, толщина 2
# Добавляем след для обучающих данных
fig.add_trace(go.Scatter(x=train_dates, y=train_target,
                         mode='lines', name='Исходные данные (обучающая выборка)',
                         line=dict(color='blue', **line_style)))
# Добавляем след для тестовых данных
fig.add_trace(go.Scatter(x=test_dates, y=test_target,
                         mode='lines', name='Исходные данные (тестовая выборка)',
                         line=dict(color='orange', **line_style)))
# Настраиваем заголовок и оси
fig.update_layout(title='Сравнение исходных значений и предсказаний',
                  xaxis_title='Дата',
                  yaxis_title='Сумма (тыс. руб)',
                  legend=dict(x=0, y=1),
                  hovermode='x unified')
# Показываем график
fig.show()

Отрисовали наш Временной ряд

Построим прогноз нейросетью LSTM

In [152]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
def prepare_data(data, look_back=1):
    scaler = MinMaxScaler(feature_range=(0, 1))
    data_scaled = scaler.fit_transform(data)

    X, Y = [], []
    for i in range(len(data_scaled) - look_back - 1):
        X.append(data_scaled[i:(i + look_back), 0])
        Y.append(data_scaled[i + look_back, 0])

    X, Y = np.array(X), np.array(Y)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    return X, Y, scaler

train_X, train_Y, scaler = prepare_data(train_target.values.reshape(-1, 1))
test_X, test_Y, _ = prepare_data(test_target.values.reshape(-1, 1), look_back=train_X.shape[1])
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))  # Выходной слой для предсказания

model.compile(optimizer='adam', loss='mse')
model.fit(train_X, train_Y, epochs=100, batch_size=32, verbose=1)

Epoch 1/100


c:\Users\Александр\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 835us/step - loss: 0.0099
Epoch 2/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 822us/step - loss: 0.0058
Epoch 3/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 800us/step - loss: 0.0032
Epoch 4/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 824us/step - loss: 0.0038 
Epoch 5/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 864us/step - loss: 0.0036
Epoch 6/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 824us/step - loss: 0.0033 
Epoch 7/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 796us/step - loss: 0.0034
Epoch 8/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 798us/step - loss: 0.0041 
Epoch 9/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 808us/step - loss: 0.0035 
Epoch 10/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 779us/step - loss: 0.0032
Epoch 11/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 772us/step - loss: 0.0041
Epoch 12/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 793us/step - loss: 0.0039
Epoch 13/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 793us/step - loss: 0.0036
Epoch 14/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 788us/step - loss: 0.0031 
Epoch 15/100
171/171 ━

In [153]:
predictions = model.predict(test_X)
predictions_inverse = scaler.inverse_transform(predictions)

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


In [154]:
print("Предсказания на тестовой выборке:")
print(predictions_inverse)

Предсказания на тестовой выборке:
[[  352558.06]
 [  352558.06]
 [  352558.06]
 [  352558.06]
 [  352558.06]
 [  352558.06]
 [ 1381996.1 ]
 [ 1381996.1 ]
 [ 1381996.1 ]
 [ 1381996.1 ]
 [ 1381996.1 ]
 [ 1381996.1 ]
 [  506231.53]
 [  506231.53]
 [  506231.53]
 [  506231.53]
 [  506231.53]
 [  506231.53]
 [  549434.  ]
 [  549434.  ]
 [  549434.  ]
 [  549434.  ]
 [  549434.  ]
 [  549434.  ]
 [  677272.9 ]
 [  677272.9 ]
 [  677272.9 ]
 [  677272.9 ]
 [  677272.9 ]
 [  677272.9 ]
 [  995486.44]
 [  995486.44]
 [  995486.44]
 [  995486.44]
 [  995486.44]
 [  995486.44]
 [ 1164653.5 ]
 [ 1164653.5 ]
 [ 1164653.5 ]
 [ 1164653.5 ]
 [ 1164653.5 ]
 [ 1164653.5 ]
 [ 1375937.5 ]
 [ 1375937.5 ]
 [ 1375937.5 ]
 [ 1375937.5 ]
 [ 1375937.5 ]
 [ 1375937.5 ]
 [  616730.7 ]
 [  616730.7 ]
 [  616730.7 ]
 [  616730.7 ]
 [  616730.7 ]
 [  616730.7 ]
 [  392383.5 ]
 [  392383.5 ]
 [  392383.5 ]
 [  392383.5 ]
 [  392383.5 ]
 [  392383.5 ]
 [  604229.3 ]
 [  604229.3 ]
 [  604229.3 ]
 [  604229.3 ]
 [  60

In [155]:
# Создаем интерактивный график
fig = go.Figure()

# Общий стиль линий
line_style = dict(width=2)  # сплошная линия, толщина 2

# Добавляем след для обучающих данных
fig.add_trace(go.Scatter(x=train_dates, y=train_target,
                         mode='lines', name='Исходные данные (обучающая выборка)',
                         line=dict(color='blue', **line_style)))

# Добавляем след для тестовых данных
fig.add_trace(go.Scatter(x=test_dates, y=test_target,
                         mode='lines', name='Исходные данные (тестовая выборка)',
                         line=dict(color='orange', **line_style)))

# Добавляем след для предсказаний
fig.add_trace(go.Scatter(x=test_dates, y=predictions_inverse.flatten(),
                         mode='lines', name='Предсказания',
                         line=dict(color='green', **line_style)))

# Настраиваем заголовок и оси
fig.update_layout(title='Сравнение исходных значений и предсказаний',
                  xaxis_title='Дата',
                  yaxis_title='Сумма (тыс. руб)',
                  legend=dict(x=0, y=1),
                  hovermode='x unified')

fig.show()

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
if len(test_target) > len(predictions_inverse.flatten()):
    test_target = test_target[:len(predictions_inverse.flatten())]
else:
    predictions_inverse = predictions_inverse[:len(test_target)]
# Расчёт MSE
mse = mean_squared_error(test_target/1000, predictions_inverse.flatten()/1000)
print(f"MSE: {mse}")
mae = mean_absolute_error(test_target/1000, predictions_inverse.flatten()/1000)
print(f"MSE: {mae}")

MSE: 466809.7116993058
MSE: 363.0681692655977


In [ ]:
df = pd.read_csv('payments_data_set.csv')
df['Дата оплаты'] = pd.to_datetime(df['Дата оплаты'])
# Сортировка DataFrame по дате платежа
df_sorted = df.sort_values(by='Дата оплаты')
df = df.dropna()
df = df.drop(df[df['прошлогодняя сумма на дату'] == 0].index)
df = df.drop(df[df['прошлогодняя сумма на день'] == 0].index)
df = df.drop(df[df['прошлогодняя недельная сумма'] == 0].index)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5755 entries, 558 to 8437
Data columns (total 13 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   Дата оплаты                        5755 non-null   datetime64[ns]
 1   Сумма (тыс. руб)                   5755 non-null   float64       
 2   День недели                        5755 non-null   int64         
 3   Номер рабочего дня в году          5755 non-null   int64         
 4   Номер недели                       5755 non-null   int64         
 5   Год                                5755 non-null   int64         
 6   Сумма платежей с начала недели     5755 non-null   float64       
 7   прошлогодняя сумма на дату         5755 non-null   float64       
 8   прошлогодняя сумма на день         5755 non-null   float64       
 9   прошлогодняя недельная сумма       5755 non-null   float64       
 10  Размер контрактуемых обязательств  5755

Теперь очередь нейросети с одним свёрточным слоем

In [158]:
def prepare_data(data, look_back=1):
    scaler = MinMaxScaler(feature_range=(0, 1))
    data_scaled = scaler.fit_transform(data)

    X, Y = [], []
    for i in range(len(data_scaled) - look_back - 1):
        X.append(data_scaled[i:(i + look_back), 0])
        Y.append(data_scaled[i + look_back, 0])

    X, Y = np.array(X), np.array(Y)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    return X, Y, scaler

train_X, train_Y, scaler = prepare_data(train_target.values.reshape(-1, 1))
test_X, test_Y, _ = prepare_data(test_target.values.reshape(-1, 1), look_back=train_X.shape[1])

In [159]:
from tensorflow.keras.layers import Conv1D, Dense, Flatten
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(train_X.shape[1], train_X.shape[2])))  # Уменьшили kernel_size до 1
model.add(Flatten())
model.add(Dense(1))  # Выходной слой для предсказания

model.compile(optimizer='adam', loss='mse')
model.fit(train_X, train_Y, epochs=100, batch_size=32, verbose=1)

Epoch 1/100


c:\Users\Александр\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 911us/step - loss: 0.0115
Epoch 2/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 906us/step - loss: 0.0040
Epoch 3/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 911us/step - loss: 0.0031
Epoch 4/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 930us/step - loss: 0.0040
Epoch 5/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 886us/step - loss: 0.0038
Epoch 6/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 924us/step - loss: 0.0044 
Epoch 7/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 895us/step - loss: 0.0036 
Epoch 8/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 919us/step - loss: 0.0039
Epoch 9/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 996us/step - loss: 0.0047
Epoch 10/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 904us/step - loss: 0.0039 
Epoch 11/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 914us/step - loss: 0.0041 
Epoch 12/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 900us/step - loss: 0.0033 
Epoch 13/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 909us/step - loss: 0.0036 
Epoch 14/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 897us/step - loss: 0.0039
Epoch 15/100
171/171 

In [160]:
predictions = model.predict(test_X)
predictions_inverse = scaler.inverse_transform(predictions)

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 


In [161]:
print("Предсказания на тестовой выборке:")
print(predictions_inverse)

Предсказания на тестовой выборке:
[[  285715.47]
 [  285715.47]
 [  285715.47]
 [  285715.47]
 [  285715.47]
 [  285715.47]
 [ 1286694.8 ]
 [ 1286694.8 ]
 [ 1286694.8 ]
 [ 1286694.8 ]
 [ 1286694.8 ]
 [ 1286694.8 ]
 [  424430.75]
 [  424430.75]
 [  424430.75]
 [  424430.75]
 [  424430.75]
 [  424430.75]
 [  467202.8 ]
 [  467202.8 ]
 [  467202.8 ]
 [  467202.8 ]
 [  467202.8 ]
 [  467202.8 ]
 [  593623.6 ]
 [  593623.6 ]
 [  593623.6 ]
 [  593623.6 ]
 [  593623.6 ]
 [  593623.6 ]
 [  907375.06]
 [  907375.06]
 [  907375.06]
 [  907375.06]
 [  907375.06]
 [  907375.06]
 [ 1073632.8 ]
 [ 1073632.8 ]
 [ 1073632.8 ]
 [ 1073632.8 ]
 [ 1073632.8 ]
 [ 1073632.8 ]
 [ 1280763.8 ]
 [ 1280763.8 ]
 [ 1280763.8 ]
 [ 1280763.8 ]
 [ 1280763.8 ]
 [ 1280763.8 ]
 [  533779.75]
 [  533779.75]
 [  533779.75]
 [  533779.75]
 [  533779.75]
 [  533779.75]
 [  317717.12]
 [  317717.12]
 [  317717.12]
 [  317717.12]
 [  317717.12]
 [  317717.12]
 [  521416.6 ]
 [  521416.6 ]
 [  521416.6 ]
 [  521416.6 ]
 [  52

In [ ]:
import plotly.graph_objects as go
import pandas as pd

fig = go.Figure()

line_style = dict(width=2)

# Добавляем след для обучающих данных
fig.add_trace(go.Scatter(x=train_dates, y=train_target,
                         mode='lines', name='Исходные данные (обучающая выборка)',
                         line=dict(color='blue', **line_style)))

# Добавляем след для тестовых данных
fig.add_trace(go.Scatter(x=test_dates, y=test_target,
                         mode='lines', name='Исходные данные (тестовая выборка)',
                         line=dict(color='orange', **line_style)))

# Добавляем след для предсказаний
fig.add_trace(go.Scatter(x=test_dates, y=predictions_inverse.flatten(),
                         mode='lines', name='Предсказания',
                         line=dict(color='green', **line_style)))

# Настраиваем заголовок и оси
fig.update_layout(title='Сравнение исходных значений и предсказаний',
                  xaxis_title='Дата',
                  yaxis_title='Сумма (тыс. руб)',
                  legend=dict(x=0, y=1),
                  hovermode='x unified')

fig.show()

In [ ]:
from sklearn.metrics import mean_squared_error

if len(test_target) > len(predictions_inverse.flatten()):
    test_target = test_target[:len(predictions_inverse.flatten())]
else:
    predictions_inverse = predictions_inverse[:len(test_target)]
# Расчёт MSE
mse = mean_squared_error(test_target/1000, predictions_inverse.flatten()/1000)
print(f"MSE: {mse}")
mae = mean_absolute_error(test_target/1000, predictions_inverse.flatten()/1000)
print(f"MSE: {mae}")

MSE: 272821.4639736646
MSE: 244.60512152350628


In [ ]:
df = pd.read_csv('payments_data_set.csv')
df['Дата оплаты'] = pd.to_datetime(df['Дата оплаты'])
df_sorted = df.sort_values(by='Дата оплаты')
df = df.dropna()
df = df.drop(df[df['прошлогодняя сумма на дату'] == 0].index)
df = df.drop(df[df['прошлогодняя сумма на день'] == 0].index)
df = df.drop(df[df['прошлогодняя недельная сумма'] == 0].index)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5755 entries, 558 to 8437
Data columns (total 13 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   Дата оплаты                        5755 non-null   datetime64[ns]
 1   Сумма (тыс. руб)                   5755 non-null   float64       
 2   День недели                        5755 non-null   int64         
 3   Номер рабочего дня в году          5755 non-null   int64         
 4   Номер недели                       5755 non-null   int64         
 5   Год                                5755 non-null   int64         
 6   Сумма платежей с начала недели     5755 non-null   float64       
 7   прошлогодняя сумма на дату         5755 non-null   float64       
 8   прошлогодняя сумма на день         5755 non-null   float64       
 9   прошлогодняя недельная сумма       5755 non-null   float64       
 10  Размер контрактуемых обязательств  5755

Проверим работу Случайного леса

In [165]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor

def prepare_data(data):
    scaler = MinMaxScaler(feature_range=(0, 1))
    data_scaled = scaler.fit_transform(data)
    return data_scaled, scaler

train_X, scaler = prepare_data(train_target.values.reshape(-1, 1))
test_X, _ = prepare_data(test_target.values.reshape(-1, 1))

In [ ]:
model = RandomForestRegressor(n_estimators=100, random_state=42)

if len(train_X) > len(train_Y):
    train_X = train_X[:len(train_Y)]
else:
    train_Y = train_Y[:len(train_X)]
model.fit(train_X, train_Y)

RandomForestRegressor(random_state=42)

In [167]:
predictions = model.predict(test_X)
predictions_inverse = scaler.inverse_transform(predictions.reshape(-1, 1)).flatten()


In [168]:
print("Предсказания на тестовой выборке:")
print(predictions_inverse)

Предсказания на тестовой выборке:
[  236568.82080714   236568.82080714   236568.82080714   236568.82080714
   236568.82080714   236568.82080714  1374419.75090194  1374419.75090194
  1374419.75090194  1374419.75090194  1374419.75090194  1374419.75090194
   387474.91481119   387474.91481119   387474.91481119   387474.91481119
   387474.91481119   387474.91481119   495525.76568189   495525.76568189
   495525.76568189   495525.76568189   495525.76568189   495525.76568189
   530583.52572083   530583.52572083   530583.52572083   530583.52572083
   530583.52572083   530583.52572083   878530.92331307   878530.92331307
   878530.92331307   878530.92331307   878530.92331307   878530.92331307
  1163623.55945677  1163623.55945677  1163623.55945677  1163623.55945677
  1163623.55945677  1163623.55945677  1355194.92204394  1355194.92204394
  1355194.92204394  1355194.92204394  1355194.92204394  1355194.92204394
   455822.72911071   455822.72911071   455822.72911071   455822.72911071
   455822.7291107

In [ ]:
import plotly.graph_objects as go
import pandas as pd

fig = go.Figure()

line_style = dict(width=2)

# Добавляем след для обучающих данных
fig.add_trace(go.Scatter(x=train_dates, y=train_target,
                         mode='lines', name='Исходные данные (обучающая выборка)',
                         line=dict(color='blue', **line_style)))

# Добавляем след для тестовых данных
fig.add_trace(go.Scatter(x=test_dates, y=test_target,
                         mode='lines', name='Исходные данные (тестовая выборка)',
                         line=dict(color='orange', **line_style)))

# Добавляем след для предсказаний
fig.add_trace(go.Scatter(x=test_dates, y=predictions_inverse,
                         mode='lines', name='Предсказания',
                         line=dict(color='green', **line_style)))

# Настраиваем заголовок и оси
fig.update_layout(title='Сравнение исходных значений и предсказаний',
                  xaxis_title='Дата',
                  yaxis_title='Сумма (тыс. руб)',
                  legend=dict(x=0, y=1),
                  hovermode='x unified')

fig.show()

In [170]:
from sklearn.metrics import mean_squared_error

# Расчёт MSE
mse = mean_squared_error(test_target/1000, predictions_inverse/1000)
print(f"MSE: {mse}")
mae = mean_absolute_error(test_target/1000, predictions_inverse.flatten()/1000)
print(f"MSE: {mae}")

MSE: 469091.04655024543
MSE: 315.17066082374294


In [171]:
df = pd.read_csv('payments_data_set.csv')
# Убедитесь, что столбец 'Дата оплаты' имеет тип datetime
df['Дата оплаты'] = pd.to_datetime(df['Дата оплаты'])
# Сортировка DataFrame по дате платежа
df_sorted = df.sort_values(by='Дата оплаты')
df = df.dropna()
df = df.drop(df[df['прошлогодняя сумма на дату'] == 0].index)
df = df.drop(df[df['прошлогодняя сумма на день'] == 0].index)
df = df.drop(df[df['прошлогодняя недельная сумма'] == 0].index)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5755 entries, 558 to 8437
Data columns (total 13 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   Дата оплаты                        5755 non-null   datetime64[ns]
 1   Сумма (тыс. руб)                   5755 non-null   float64       
 2   День недели                        5755 non-null   int64         
 3   Номер рабочего дня в году          5755 non-null   int64         
 4   Номер недели                       5755 non-null   int64         
 5   Год                                5755 non-null   int64         
 6   Сумма платежей с начала недели     5755 non-null   float64       
 7   прошлогодняя сумма на дату         5755 non-null   float64       
 8   прошлогодняя сумма на день         5755 non-null   float64       
 9   прошлогодняя недельная сумма       5755 non-null   float64       
 10  Размер контрактуемых обязательств  5755

Финальная модель - XGBoost

In [172]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb

def prepare_data(data):
    scaler = MinMaxScaler(feature_range=(0, 1))
    data_scaled = scaler.fit_transform(data)
    return data_scaled, scaler

train_X, scaler = prepare_data(train_target.values.reshape(-1, 1))
test_X, _ = prepare_data(test_target.values.reshape(-1, 1))

In [ ]:
# Обучение модели
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1)
if len(train_X) > len(train_Y):
    train_X = train_X[:len(train_Y)]
else:
    train_Y = train_Y[:len(train_X)]
model.fit(train_X, train_Y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.1, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=100,
             n_jobs=None, num_parallel_tree=None, ...)

In [174]:
predictions = model.predict(test_X)
predictions_inverse = scaler.inverse_transform(predictions.reshape(-1, 1)).flatten()

In [ ]:

fig = go.Figure()

line_style = dict(width=2)

# Добавляем след для обучающих данных
fig.add_trace(go.Scatter(x=train_dates, y=train_target,
                         mode='lines', name='Исходные данные (обучающая выборка)',
                         line=dict(color='blue', **line_style)))

# Добавляем след для тестовых данных
fig.add_trace(go.Scatter(x=test_dates, y=test_target,
                         mode='lines', name='Исходные данные (тестовая выборка)',
                         line=dict(color='orange', **line_style)))

# Добавляем след для предсказаний
fig.add_trace(go.Scatter(x=test_dates, y=predictions_inverse,
                         mode='lines', name='Предсказания',
                         line=dict(color='green', **line_style)))

# Настраиваем заголовок и оси
fig.update_layout(title='Сравнение исходных значений и предсказаний',
                  xaxis_title='Дата',
                  yaxis_title='Сумма (тыс. руб)',
                  legend=dict(x=0, y=1),
                  hovermode='x unified')

fig.show()

In [176]:
from sklearn.metrics import mean_squared_error

# Расчёт MSE
mse = mean_squared_error(test_target/1000, predictions_inverse/1000)
print(f"MSE: {mse}")
mae = mean_absolute_error(test_target/1000, predictions_inverse.flatten()/1000)
print(f"MSE: {mae}")

MSE: 258245.49398183456
MSE: 335.14833489768444
